In [1]:
import numpy as np
from ondelette import *
import tensorflow as tf
import tensorflow_io as tfio

dtype = tf.float32
rng = np.random.default_rng()

2025-03-08 15:53:57.087460: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-08 15:53:57.093815: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-08 15:53:57.114259: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-08 15:53:57.148711: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-08 15:53:57.148758: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-08 15:53:57.207178: I tensorflow/core/platform/cpu_feature_guard.cc:

In [3]:
samples_np = rng.random(size=(1, 12), dtype=np.float32)
samples = tf.constant(samples_np)
print(f'SAMPLES: {samples.shape}')
print(samples.numpy())

for name, make in wavelets.all.items():
    wavelet = make(dtype=dtype)
    wavelet_width = wavelet.shape[0]
    print(f'{name} (w={wavelet_width}):')
    filters = wavelets.dwt_filters(wavelet)
    inverse_filters = wavelets.idwt_filters(wavelet)

    dec = wavelets.wavedec(filters, 2, samples)
    rec = wavelets.waverec(inverse_filters, dec)
    print(f'RECONSTRUCTION: {rec.shape}')
    print(rec.numpy())


SAMPLES: (1, 12)
[[0.7739503  0.43203986 0.06420648 0.4537459  0.73903835 0.51343024
  0.83945215 0.5941224  0.30912483 0.15395784 0.95479554 0.52599853]]
bior1.1 (w=2):
Level 1: 0 + 3 + 1 = 4
Level 2: 0 + 6 + 2 = 8
RECONSTRUCTION: (1, 16)
[[ 7.7395016e-01  4.3203977e-01  6.4206392e-02  4.5374581e-01
   7.3903823e-01  5.1343012e-01  8.3945203e-01  5.9412229e-01
   3.0912477e-01  1.5395780e-01  9.5479548e-01  5.2599847e-01
  -7.3945965e-09 -7.3945965e-09 -7.3945965e-09 -7.3945965e-09]]
bior1.3 (w=6):
Level 1: 4 + 6 + 0 = 10
Level 2: 12 + 8 + 0 = 20
RECONSTRUCTION: (1, 36)
[[ 0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00
   0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00
   0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00
   0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00
   0.0000000e+00  0.0000000e+00 -9.1186730e-10 -9.1186730e-10
  -9.1186730e-10 -9.1186730e-10 -2.1119616e-08 -2.1119616e-08
   7.7395022e-01  4.3203980e-01  6.4206466e-02  4.